**本notebook为选做部分**

# Transformer with Encoder and Decoder

## 0. 准备


推荐的视频资料：
1.  [台大李宏毅机器学习2022春](https://speech.ee.ntu.edu.tw/~hylee/ml/2022-spring.php)

2. [李沐：Transformer论文逐段精读](https://www.bilibili.com/video/BV1pu411o7BE)

有疑问？先在[Q&A](./Q&A/transformer.md)中看看有没有类似问题。

导入本实验所需的所有环境和库。

In [ ]:
! pip install tqdm

In [1]:
import os
import re
from tqdm import tqdm

import mindspore
from mindspore import nn
from mindspore import ops
from mindspore import Tensor
from mindspore import dtype as mstype
from mindspore import numpy as mnp
from mindspore import save_checkpoint
from mindspore import load_checkpoint, load_param_into_net
from mindspore.train import BleuScore



## 1. Transformer简介
Transformer是一种神经网络结构，由Vaswani等人在2017年的论文[Attention Is All You Need](./assets/Attention_Is_All_You_Need.pdf)中提出，用于处理机器翻译、语言建模和文本生成等自然语言处理任务。

Transformer与传统NLP特征提取类模型的区别主要在以下两点：

- Transformer是一个纯基于注意力机制的结构，并将自注意力机制和多头注意力机制的概念运用到模型中；
- 由于缺少RNN模型的时序性，Transformer引入了位置编码，在数据上而非模型中添加位置信息；

<div align=center><img src="./assets/images/2024-03-25-23-20-48.png" alt="attention"></div>

以上的处理带来了几个优点：
- 更容易并行化，训练更加高效；
- 在处理长序列的任务中表现优秀，可以快速捕捉**长距离**中的关联信息。

本实验将从Transformer的基本结构开始，逐步深入，带领大家了解Transformer的原理和实现。

## 2. Transformer的基本结构

Transformer的基本结构主要包括Encoder和Decoder两部分，如下图所示：

<div align=center><img src="assets/images/2024-03-25-23-22-27.png" alt="attention"></div>

这里的“encoder”和“decoder”是由无数个同样结构的encoder层和decoder层堆叠组成。

在进行机器翻译时，encoder解读源语句（被翻译的句子）的信息，并传输给decoder。decoder接收源语句信息后，结合当前输入（目前翻译的情况），预测下一个单词，直到生成完整的句子。

<div align=center><img src="assets/images/2024-03-25-23-25-07.png" alt="attention"></div>

## 3. 词嵌入（word embedding）和位置编码（position encoding）

在Transformer中，词嵌入和位置编码是输入模型的两个重要步骤。它解决的问题是：如何将输入的单词序列转换为模型可以处理的向量形式。


### word embedding

word2vec等一系列技术将单词映射到一个固定长度的向量空间，这种映射被称为词嵌入。词嵌入的作用是将单词映射到一个连续的向量空间，使得单词之间的语义关系可以在向量空间中得到体现。

比如，我们可以将“king”和“queen”映射到向量空间中，然后通过向量空间中的运算，我们可以得到“king”-“man”+“woman”=“queen”。

什么是好的词向量映射呢？一个好的词向量映射应该满足以下两个条件：

- 语义相近的单词在向量空间中距离较近；
- 向量空间中的运算能够体现单词之间的语义关系。

很明显，越多的词进行训练，学到的词向量嵌入就会越好。由于数据量有限，本实验中我们将采用ms框架中训练好的embedding层：

```python
nn.Embedding(src_vocab_size, d_model)
```

### position encoding(PE)
Transformer模型不包含RNN的循环结构，所以无法在模型中记录时序信息，这样会导致attention机制无法识别由顺序改变而产生的句子含义的改变，如“我爱机器学习”和“机器爱学习我”。

为了弥补这个缺陷，我们选择在输入数据中额外添加表示位置信息的位置编码。

位置编码$PE$的形状与经过word embedding后的输出$X$相同，对于索引为[pos, 2i]的元素，以及索引为[pos, 2i+1]的元素，位置编码的计算如下：

$$PE_{(pos,2i)} = \sin\Bigg(\frac{pos}{10000^{2i/d_{\text{model}}}}\Bigg)$$

$$PE_{(pos,2i+1)} = \cos\Bigg(\frac{pos}{10000^{2i/d_{\text{model}}}}\Bigg)$$

在下面的代码中，我们实现了位置编码，输入经过word embedding后的结果$X$，输出添加位置信息后的结果$X + PE$。

In [4]:
class PositionalEncoding(nn.Cell):
    """位置编码"""

    def __init__(self, d_model, dropout_p=0.1, max_len=100):
        super().__init__()

        # p (Union[float, int, None]) - 输入神经元丢弃率，数值范围介于[0, 1)之间。例如，p =0.9，删除90%的神经元。默认值：None。
        self.dropout = nn.Dropout(p = dropout_p)

        # 位置信息
        # pe: [max_len, d_model]
        self.pe = ops.Zeros()((max_len, d_model), mstype.float32)

        # pos: [max_len, 1]
        # angle: [d_model/2, ]
        # pos/angle: [max len, d_model/2]
        """修改这部分代码，实现位置编码"""
        pos = ...
        angle = ...
        # pe: [max len, d_model]
        self.pe[:, 0::2] = ...
        self.pe[:, 1::2] = ...
        """--------------------------"""

    def construct(self, x):
        batch_size = x.shape[0]

        # broadcast
        # pe: [batch_size, max_len, d_model]
        pe = self.pe.expand_dims(0)
        pe = ops.broadcast_to(pe, (batch_size, -1, -1))

        # 将位置编码截取至x同等大小
        # x: [batch_size, seq_len, d_model]
        x = x + pe[:, :x.shape[1], :]
        return self.dropout(x)

In [5]:
## test

def test_positional_encoding():
    x = ops.Zeros()((1, 2, 4), mstype.float32)
    pe = PositionalEncoding(4)
    print(pe(x))

test_positional_encoding()

[[[0.         1.         0.         1.        ]
  [0.84147096 0.5403023  0.00999983 0.99995   ]]]


## 4.0 Attention!

在Transformer中，attention机制是核心。它的作用是计算输入序列中每个单词对于当前单词的重要性，然后根据重要性对输入序列进行加权求和。
注意力机制便是在判断**词在句子中的重要性**，我们通过**注意力分数**来表达某个词在句子中的重要性，分数越高，说明该词对完成该任务的重要性越大。

计算注意力分数时，我们主要参考三个因素：**query**、**key**和**value**。

- `query`：任务内容
- `key`：索引/标签（帮助定位到答案）
- `value`：答案

在上面的例子中，如“情感分类”、“电影名字”、“中译英”等为`query`，每次对于任务内容的回答即为`value`。至于什么是`key`， 用一个比较直观的举例来说，每次登录视频网站搜索视频时，搜索的内容为`query`，搜索结果中显示的视频名称为`key`，它与任务内容相关，并可以引导我们至具体的视频内容（`value`）。
一般在文本翻译中，我们希望翻译后的句子的意思和原始句子相似，所以进行注意力分数计算时，`query`一般和目标序列，即翻译后的句子有关，`key`则与源序列，即翻译前的原始句子有关。


计算注意力分数，即为计算`query`与`key`的相似度，也就是计算两个向量距离有多近。常用的计算注意力分数的方式有两种：`additive attention`和`scaled dot-product attention`，在这里我们主要介绍第二种方法。

在几何角度，点积（dot product）表示一个向量在另一个向量方向上的投影。换句话说，从几何角度上解读，点积代表了某个向量中的多少是和另一个向量相似的。

将这个概念运用到当前的情境中，我们想要求`query`和`key`之间有多少是相似的，则需要计算`query`和`key`的点积。

同时，为了避免`query`（$Q \in R^{n\times d_{model}}$）和`key`($K \in R^{m\times d_{model}}$)本身的“大小”影响到相似度的计算，我们需要在点乘后除以$\sqrt{d_{model}}$。[1](#附录)

$$\text{Attention Score}(Q, K)=\frac{QK^T}{\sqrt{d_{model}}}$$

我们将该相似度的区间限制与0到1之间，并令其作用在`value`上。

$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_{model}}}\right)V$$

在如下代码中，我们需要实现scaled dot-product attention的计算， 调用类后，返回的是加权后的value（output）以及注意力权重（attn）。

In [9]:
class ScaledDotProductAttention(nn.Cell):
    def __init__(self, dropout_p=0.):
        super().__init__()
        self.softmax = nn.Softmax()
        self.dropout = nn.Dropout(p=dropout_p)
        self.sqrt = ops.Sqrt()


    def construct(self, query, key, value, attn_mask=None):
        """scaled dot product attention"""
        # 计算scaling factor
        embed_size = query.shape[-1]
        scaling_factor = self.sqrt(Tensor(embed_size, mstype.float32))
        
        # 注意力权重计算
        # 计算query和key之间的点积，并除以scaling factor进行归一化
        """
        code here
        """

        # 注意力掩码机制
        if attn_mask is not None:
            attn = attn.masked_fill(attn_mask, -1e9)

        # softmax，保证注意力权重范围在0-1之间
        attn = self.softmax(attn)

        # dropout
        attn = self.dropout(attn)

        # 对value进行加权
        output = ops.matmul(attn, value)

        return (output, attn)

## 4.1 self-attention

介绍完attention机制，接下来说说如何将它用在我们的任务当中。

将attention中的`query`, `key`和`value`都变成句子本身。这就是“自”注意力机制（self-attention）。
自注意力机制中，我们关注句子本身，查看每个单词对于周边单词的重要性。这样可以很好地理清句子中的逻辑关系，如代词指代。
举个例子，在'`The animal` didn't cross the street because `it` was too tired'这句话中，'it'指代句中的'The animal'，所以自注意力会赋予'The'、'animal'更高的注意力分值。

所以情况变成：给定序列$X \in \mathbb{R}^{n \times d_{model}}$，序列长度为$n$，维度为$d_{model}$。在计算自注意力时，$Q = K = V = X$

$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_{model}}}\right)V = \text{softmax}\left(\frac{XX^T}{\sqrt{d_{model}}}\right)X$$

其中，序列中位置为$i$的词与位置为$j$的词之间的自注意力分数为：

$$\text{Attention}(Q, K, V)_{i,j} = \frac{\text{exp}\left(\frac{Q_iK_j^T}{\sqrt{d_{model}}}\right)}{\sum_{k=1}^{n}\text{exp}\left(\frac{Q_iK_k^T}{\sqrt{d_{model}}}\right)}V_j$$

正因如此，我们只需要将上面实现的attention机制，用同样的参数代入就可以了。在这部分，你不需要做任何改动。

## 4.2 Multi-head attention

在实际应用中，我们不仅仅使用一个注意力头，而是使用多个注意力头。这样可以让模型学习到不同的注意力权重，从而更好地捕捉到不同的语义信息。举个上面提到的例子：

'`The animal` didn't cross the street because `it` was too tired'

正所谓一心不能二用，一个注意力头很可能只关注到了it和animal之间的关系，那假设我们的模型是一个有很多个头的巨大怪兽，另一个注意力头可能会关注到animal和cross之间的关系，或者是animal和street之间的关系。
![](assets/images/2024-03-26-00-28-36.png)

    那么问题来了，我们输入同一个句子embedding的向量，理论上每个头接受的参数相同，学到的都是一样的，那多头就没有用了？其实不然，多头注意力的优势在于，每个头都有自己的权重矩阵，这样就可以学到不同的语义信息，从而更好地捕捉到不同的语义信息。

多头注意力通过对输入的embedding乘以不同的权重参数$W^{Q}$、$W^{K}$和$W^{V}$，将其映射到多个小维度空间中，我们称之为“头”（head），每个头部会**并行**计算自己的自注意力分数。

$$\text{head}_i = \text{Attention}(QW^Q_i, KW^K_i, VW^V_i) = \text{softmax}\left(\frac{Q_iK_i^T}{\sqrt{d_{k}}}\right)V_i$$

$W^Q_i \in \mathbb{R}^{d_{model}\times d_{k}}$、$W^K_i \in \mathbb{R}^{d_{model}\times d_{k}}$和$W^V_i \in \mathbb{R}^{d_{model}\times d_{v}}$为可学习的权重参数。一般为了平衡计算成本，我们会取$d_k = d_v = d_{model} / n_{head}$。

在获得多组自注意力分数后，我们将结果拼接到一起，得到多头注意力的最终输出。$W^O$为可学习的权重参数，用于将拼接后的多头注意力输出映射回原来的维度。


$$\text{MultiHead}(Q, K, V)=\text{Concat}(\text{head}_1, ..., \text{head}_h)W^O$$

简单来说，在多头注意力中，每个头部可以'解读'输入内容的不同方面，比如：捕捉全局依赖关系、关注特定语境下的词元、识别词和词之间的语法关系等。

现在，你已经清楚了多头注意力的设计了，快来实现一个吧！

In [10]:
class MultiHeadAttention(nn.Cell):
    def __init__(self, d_model, d_k, n_heads, dropout_p=0.):
        super().__init__()
        self.n_heads = n_heads
        self.d_k = d_k
        self.W_Q = nn.Dense(d_model, d_k * n_heads)
        self.W_K = nn.Dense(d_model, d_k * n_heads)
        self.W_V = nn.Dense(d_model, d_k * n_heads)
        self.W_O = nn.Dense(n_heads * d_k, d_model)
        self.attention = ScaledDotProductAttention(dropout_p=dropout_p)

    def construct(self, query, key, value, attn_mask):
        """
        query: [batch_size, len_q, d_model]
        key: [batch_size, len_k, d_model]
        value: [batch_size, len_k, d_model]
        attn_mask: [batch_size, seq_len, seq_len]
        """

        batch_size = query.shape[0]

        # 将query，key和value分别乘以对应的权重，并分割为不同的“头”
        # q_s: [batch_size, len_q, n_heads, d_k]
        # k_s: [batch_size, len_k, n_heads, d_k]
        # v_s: [batch_size, len_k, n_heads, d_k]
        """
        code here
        """

        # 调整query，key和value的维度
        # q_s: [batch_size, n_heads, len_q, d_k]
        # k_s: [batch_size, n_heads, len_k, d_k]
        # v_s: [batch_size, n_heads, len_k, d_k]
        q_s = q_s.transpose((0, 2, 1, 3))
        k_s = k_s.transpose((0, 2, 1, 3))
        v_s = v_s.transpose((0, 2, 1, 3))

        # attn_mask的dimension需与q_s, k_s, v_s对应
        # attn_mask: [batch_size, n_heads, seq_len, seq_len]
        attn_mask = attn_mask.expand_dims(1)
        attn_mask = ops.tile(attn_mask, (1, self.n_heads, 1, 1))

        # 计算每个头的注意力分数
        # context: [batch_size, n_heads, len_q, d_k]
        # attn: [batch_size, n_heads, len_q, len_k]
        context, attn = self.attention(q_s, k_s, v_s, attn_mask)

        # concatenate
        # context: [batch_size, len_q, n_heads * d_k]
        context = context.transpose((0, 2, 1, 3)).view((batch_size, -1, self.n_heads * self.d_k))

        # 乘以W_O
        # output: [batch_size, len_q, n_heads * d_k]
        output = self.W_O(context)

        return output, attn

In [11]:
dmodel, dk, nheads = 10, 2, 5
q = k = v = ops.ones((1, 2, 10), mstype.float32)
attn_mask = Tensor([False]).broadcast_to((1, 2, 2))
multi_head_attn = MultiHeadAttention(dmodel, dk, nheads)
output, attn = multi_head_attn(q, k, v, attn_mask)
print(output.shape, attn.shape)

(1, 2, 10) (1, 5, 2, 2)


## 4.3 attention padding mask

在实际应用中，我们的输入序列长度是不固定的，但是在进行矩阵运算时，我们需要保证输入的矩阵是固定的，这就需要对输入序列进行padding操作。我们为了统一模型输入的长度，使用\<pad\>占位符补齐一些稍短的文本。

```text
处理前："Hello world!"
处理后：<bos> hello world ! <eos> <pad> <pad>
```

这些\<pad\>占位符没有任何意义，不应该参与注意力分数计算中。为此我们在注意力中加入了padding掩码，即识别输入序列中的\<pad\>占位符，保证计算时这些位置对应的注意力分数为0。

实现一下吧：

In [18]:
def get_attn_pad_mask(seq_q, seq_k, pad_idx):
    """注意力掩码：识别序列中的<pad>占位符

    Args:
        seq_q (Tensor): query序列，shape = [batch size, query len]
        seq_k (Tensor): key序列，shape = [batch size, key len]
        pad_idx (Tensor): key序列<pad>占位符对应的数字索引
    """
    batch_size, len_q = seq_q.shape
    batch_size, len_k = seq_k.shape

    # 如果序列中元素对应<pad>占位符，则该位置在mask中对应元素为True
    # pad_attn_mask: [batch size, key len]
    pad_attn_mask = ops.equal(seq_k, pad_idx)

    # 增加额外的维度
    # pad_attn_mask: [batch size, 1, key len]
    pad_attn_mask = pad_attn_mask.expand_dims(1)
    # 将掩码广播到[batch size, query len, key len]
    pad_attn_mask = ops.broadcast_to(pad_attn_mask, (batch_size, len_q, len_k))

    return pad_attn_mask

In [19]:
# test
q = k = Tensor([[1, 1, 0, 0]], mstype.float32)
pad_idx = 0
mask = get_attn_pad_mask(q, k, pad_idx)
print(mask)

[[[False False  True  True]
  [False False  True  True]
  [False False  True  True]
  [False False  True  True]]]


## 5. Add & norm

Add & Norm层本质上是残差连接后紧接了一个LayerNorm层。

$\text{Add\&Norm}(x) = \text{LayerNorm}(x + \text{Sublayer}(x))$

- Add：残差连接，帮助缓解网络退化问题，注意需要满足$x$与$\text{SubLayer}(x)的形状一致$；
- Norm：Layer Norm，层归一化，帮助模型更快地进行收敛；

In [14]:
class AddNorm(nn.Cell):
    def __init__(self, d_model, dropout_p=0.):
        super().__init__()
        self.layer_norm = nn.LayerNorm((d_model, ), epsilon=1e-5)
        self.dropout = nn.Dropout(p=dropout_p)
    
    def construct(self, x, residual):
        return self.layer_norm(self.dropout(x) + residual)

In [15]:
x = ops.ones((1, 2, 4), mstype.float32)
residual = ops.ones((1, 2, 4), mstype.float32)
add_norm = AddNorm(4)
print(add_norm(x, residual).shape)

(1, 2, 4)


## 6. 基于位置的前馈神经网络 （Position-Wise Feed-Forward Network）

基于位置的前馈神经网络被用来对输入中的每个位置进行非线性变换。它由两个线性层组成，层与层之间需要经过ReLU激活函数。

$$\mathrm{FFN}(x) = \mathrm{ReLU}(xW_1 + b_1)W_2 + b_2$$

相比固定的ReLU函数，基于位置的前馈神经网络可以处理更加复杂的关系，并且由于前馈网络是基于位置的，可以捕获到不同位置的信息，并为每个位置提供不同的转换。

In [16]:
class PoswiseFeedForward(nn.Cell):
    def __init__(self, d_ff, d_model, dropout_p=0.):
        super().__init__()
        self.linear1 = nn.Dense(d_model, d_ff)
        self.linear2 = nn.Dense(d_ff, d_model)
        self.dropout = nn.Dropout(p = dropout_p)
        self.relu = nn.ReLU()

    def construct(self, x):
        """
        #前馈神经网络的结构为
        1. 线性层
        2. relu
        3. dropout
        4. 线性层
        """
        x = self.linear1(x)
        x = self.relu(x)
        x = self.dropout(x)
        return self.linear2(x)


In [17]:
x = ops.ones((128, 32, 512), mstype.float32)
ffn = PoswiseFeedForward(2048, 512)
print(ffn(x).shape)

(128, 32, 512)


## 7. 拼出第一个Layer

In [20]:
class EncoderLayer(nn.Cell):
    def __init__(self, d_model, n_heads, d_ff, dropout_p=0.):
        super().__init__()
        d_k = d_model // n_heads
        if d_k * n_heads != d_model:
            raise ValueError(f"The `d_model` {d_model} can not be divisible by `num_heads` {n_heads}.")
        self.enc_self_attn = MultiHeadAttention(d_model, d_k, n_heads, dropout_p)
        self.pos_ffn = PoswiseFeedForward(d_ff, d_model, dropout_p)
        self.add_norm1 = AddNorm(d_model, dropout_p)
        self.add_norm2 = AddNorm(d_model, dropout_p)

    def construct(self, enc_inputs, enc_self_attn_mask):
        # self-attention
        enc_outputs, attn = self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs, enc_self_attn_mask)
        enc_outputs = self.add_norm1(enc_outputs, enc_inputs)

        # feed-forward
        enc_outputs = self.add_norm2(self.pos_ffn(enc_outputs), enc_outputs)
        return enc_outputs, attn

In [21]:
x = ops.ones((1, 2, 8), mstype.float32)
mask = Tensor([False]).broadcast_to((1, 2, 2))
encoder_layer = EncoderLayer(8, 4, 16)
output, attn = encoder_layer(x, mask)
print(output.shape, attn.shape)

(1, 2, 8) (1, 4, 2, 2)


## 8. Encoder!

将上面实现的encoder层堆叠`n_layers`次，并添加wording embedding与positional encoding, 就构成了一个简单的Encoder层。

<div align=center><img src="assets/images/2024-03-26-01-02-34.png" alt="attention"></div>



In [22]:
class Encoder(nn.Cell):
    def __init__(self, src_vocab_size, d_model, n_heads, d_ff, n_layers, dropout_p=0.):
        super().__init__()
        self.src_emb = nn.Embedding(src_vocab_size, d_model)
        self.pos_emb = PositionalEncoding(d_model, dropout_p)
        self.layers = nn.CellList([EncoderLayer(d_model, n_heads, d_ff, dropout_p) for _ in range(n_layers)])
        self.scaling_factor = ops.Sqrt()(Tensor(d_model, mstype.float32))

        
    def construct(self, enc_inputs, src_pad_idx):
        """enc_inputs : [batch_size, src_len]
        """
        # 将输入转换为embedding，并添加位置信息
        # enc_outputs: [batch_size, src_len, d_model]
        enc_outputs = self.src_emb(enc_inputs.astype(mstype.int32))
        enc_outputs = self.pos_emb(enc_outputs * self.scaling_factor)

        # 输入的padding掩码
        # enc_self_attn_mask: [batch_size, src_len, src_len]
        enc_self_attn_mask = get_attn_pad_mask(enc_inputs, enc_inputs, src_pad_idx)

        # 堆叠encoder层
        # enc_outputs: [batch_size, src_len, d_model]
        # enc_self_attns: [batch_size, n_heads, src_len, src_len]
        enc_self_attns = []
        for layer in self.layers:
            enc_outputs, enc_self_attn = layer(enc_outputs, enc_self_attn_mask)
            enc_self_attns.append(enc_self_attn)
        return enc_outputs, enc_self_attns

## 9. Decoder!

解码器将编码器输出的上下文序列转换为目标序列的预测结果$\hat{Y}$，该输出将在模型训练中与真实目标输出$Y$进行比较，计算损失。

<div align=center><img src="assets/images/2024-03-26-01-04-44.png" alt="attention"></div>

不同于编码器，每个Decoder层中包含两层多头注意力机制,并在最后多出一个线性层，输出对目标序列的预测结果。

- 第一层：计算目标序列的注意力分数的**掩码多头自注意力**；
- 第二层：用于计算上下文序列与目标序列对应关系，其中Decoder掩码多头注意力的输出作为query，Encoder的输出（上下文序列）作为key和value；

为什么这么设计？

想想只用如果decoder采用encoder的设计会有什么问题。很明显，decoder偷看到了下一时刻的正确输出！在训练过程中，t时刻的模型只应“观察”直到t-1时刻的所有词元，后续的词语不应该一并输入Decoder中。这就好比，写作业时，不应该对着答案参考着写，因为在实际预测任务的时候，并不会有答案喂给模型的。

### 9.1 masked multi-head attention

为了保证在t时刻，只有t-1个词元作为输入参与多头注意力分数的计算，我们需要在第一个多头注意力中额外增加一个时间mask，使目标序列中的词随时间发展逐个被暴露出来。

该注意力mask可通过三角矩阵实现，对角线以上的词元表示为不参与注意力计算的词元，标记为1。

$$\begin{matrix}
0 & 1 & 1 & 1 & 1\\
0 & 0 & 1 & 1 & 1\\
0 & 0 & 0 & 1 & 1\\
0 & 0 & 0 & 0 & 1\\
0 & 0 & 0 & 0 & 0\\
\end{matrix}$$

这一mask称为subsequent mask.

最后，将subsequent mask和padding mask合并为一个整体的掩码，确保模型既不会注意到t时刻以后的词元，也不会关注为\<pad\>的词元。

In [23]:
def get_attn_subsequent_mask(seq_q, seq_k):
    """生成时间掩码，使decoder在第t时刻只能看到序列的前t-1个元素
    
    Args:
        seq_q (Tensor): query序列，shape = [batch size, len_q]
        seq_k (Tensor): key序列，shape = [batch size, len_k]
    """
    batch_size, len_q = seq_q.shape
    batch_size, len_k = seq_k.shape
    # 生成三角矩阵
    # subsequent_mask: [batch size, len_q, len_k]
    ones = ops.ones((batch_size, len_q, len_k), mindspore.float32)
    subsequent_mask = mnp.triu(ones, k=1)
    return subsequent_mask

In [24]:
q = k = ops.ones((1, 4), mstype.float32)
mask = get_attn_subsequent_mask(q, k)
print(mask)

[[[0. 1. 1. 1.]
  [0. 0. 1. 1.]
  [0. 0. 0. 1.]
  [0. 0. 0. 0.]]]


### 一个decoder层

In [25]:
class DecoderLayer(nn.Cell):
    def __init__(self, d_model, n_heads, d_ff, dropout_p=0.):
        super().__init__()
        d_k = d_model // n_heads
        if d_k * n_heads != d_model:
            raise ValueError(f"The `d_model` {d_model} can not be divisible by `num_heads` {n_heads}.")
        self.dec_self_attn = MultiHeadAttention(d_model, d_k, n_heads, dropout_p)
        self.dec_enc_attn = MultiHeadAttention(d_model, d_k, n_heads, dropout_p)
        self.pos_ffn = PoswiseFeedForward(d_ff, d_model, dropout_p)
        self.add_norm1 = AddNorm(d_model, dropout_p)
        self.add_norm2 = AddNorm(d_model, dropout_p)
        self.add_norm3 = AddNorm(d_model, dropout_p)

    def construct(self, dec_inputs, enc_outputs, dec_self_attn_mask, dec_enc_attn_mask):
        # self-attention
        dec_outputs, dec_self_attn = self.dec_self_attn(dec_inputs, dec_inputs, dec_inputs, dec_self_attn_mask)
        dec_outputs = self.add_norm1(dec_outputs, dec_inputs)

        # encoder-decoder attention
        dec_outputs, dec_enc_attn = self.dec_enc_attn(dec_outputs, enc_outputs, enc_outputs, dec_enc_attn_mask)
        dec_outputs = self.add_norm2(dec_outputs, dec_inputs)

        # feed-forward
        dec_outputs = self.add_norm3(self.pos_ffn(dec_outputs), dec_outputs)
        return dec_outputs, dec_self_attn, dec_enc_attn

In [26]:
x = y = ops.ones((1, 2, 4), mstype.float32)
mask1 = mask2 = Tensor([False]).broadcast_to((1, 2, 2))
decoder_layer = DecoderLayer(4, 1, 16)
output, attn1, attn2 = decoder_layer(x, y, mask1, mask2)
print(output.shape, attn1.shape, attn2.shape)

(1, 2, 4) (1, 1, 2, 2) (1, 1, 2, 2)


### 一叠decoder层

将上面实现的DecoderLayer堆叠`n_layer`次，添加word embedding与positional encoding，以及最后的线性层。

输出的`dec_outputs`为对目标序列的预测

In [27]:
class Decoder(nn.Cell):
    def __init__(self, trg_vocab_size, d_model, n_heads, d_ff, n_layers, dropout_p=0.):
        super().__init__()
        self.trg_emb = nn.Embedding(trg_vocab_size, d_model)
        self.pos_emb = PositionalEncoding(d_model, dropout_p)
        self.layers = nn.CellList([DecoderLayer(d_model, n_heads, d_ff) for _ in range(n_layers)])
        self.projection = nn.Dense(d_model, trg_vocab_size)
        self.scaling_factor = ops.Sqrt()(Tensor(d_model, mstype.float32))      
        
    def construct(self, dec_inputs, enc_inputs, enc_outputs, src_pad_idx, trg_pad_idx):
        """
        dec_inputs: [batch_size, trg_len]
        enc_inputs: [batch_size, src_len]
        enc_outputs: [batch_size, src_len, d_model]
        """
        # 将输入转换为Embedding，并添加位置信息
        # dec_outputs: [batch_size, trg_len, d_model]
        dec_outputs = self.trg_emb(dec_inputs.astype(mstype.int32))
        dec_outputs = self.pos_emb(dec_outputs * self.scaling_factor)

        # decoder中自注意力的掩码
        # dec_self_attn_mask: [batch_size, trg_len, trg_len]
        dec_self_attn_pad_mask = get_attn_pad_mask(dec_inputs, dec_inputs, trg_pad_idx)
        dec_self_attn_subsequent_mask = get_attn_subsequent_mask(dec_inputs, dec_inputs)
        dec_self_attn_mask = ops.gt((dec_self_attn_pad_mask + dec_self_attn_subsequent_mask), 0)

        # encoder-decoder中的注意力padding掩码
        # dec_enc_attn_mask: [batch_size, trg_len, src_len]
        dec_enc_attn_mask = get_attn_pad_mask(dec_inputs, enc_inputs, src_pad_idx)

        # 堆叠decoder层
        # dec_outputs: [batch_size, trg_len, d_model]
        dec_self_attns, dec_enc_attns = [], []
        for layer in self.layers:
            dec_outputs, dec_self_attn, dec_enc_attn = layer(dec_outputs, enc_outputs, dec_self_attn_mask, dec_enc_attn_mask)
            dec_self_attns.append(dec_self_attn)
            dec_enc_attns.append(dec_enc_attn)

        # 线性层
        # dec_outputs: [batch_size, trg_len, trg_vocab_size]
        dec_outputs = self.projection(dec_outputs)
        return dec_outputs, dec_self_attns, dec_enc_attns

## 10. finally, Transformer!

In [28]:
class Transformer(nn.Cell):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        
    def construct(self, enc_inputs, dec_inputs, src_pad_idx, trg_pad_idx):
        """
        enc_inputs: [batch_size, src_len]
        dec_inputs: [batch_size, trg_len]
        """
        # encoder，输出表示源序列信息tensor
        # enc_ouputs: [batch_size, src_len, d_model]
        enc_outputs, enc_self_attns = self.encoder(enc_inputs, src_pad_idx)

        # decoder
        # de_outputs: [batch_size, trg_len, trg_vocab_size]
        """
        code here
        """


        # decoder logits
        # dec_logits: [batch_size * trg_len, trg_vocab_size]
        dec_logits = dec_outputs.view((-1, dec_outputs.shape[-1]))

        return dec_logits, enc_self_attns, dec_self_attns, dec_enc_attns
        

# 应用到实际任务中——机器翻译

实现并测试了你的第一个Transformer，你一定迫不及待的想试试效果吧！我们准备了一个机器翻译的任务供你尝试。

这一任务有如下的Pipeline:
1. 数据预处理： 将图像、文本等数据处理为可以计算的Tensor
2. 模型构建： 使用框架API， 搭建模型
3. 模型训练： 定义模型**训练逻辑**， 遍历**训练集**进行训练
4. 模型评估： 使用训练好的模型， 在**测试集**评估效果
5. 模型推理： 将训练好的模型部署， 输入新数据获得预测结果

In [30]:
train_path = './dataset/train'
valid_path = './dataset/valid'
test_path = './dataset/test'

## 1. 数据预处理

我们本次使用的数据集为**Multi30K数据集**，它是一个大规模的图像-文本数据集，包含30K+图片，每张图片对应英语描述，及对应的德语翻译。特别的，我们已处理好了这些英文-德文对，可以直接来使用。在本次文本翻译任务中，德语是源语言（source languag），英语是目标语言（target language）。

### 加载数据集

加载数据集，并进行分词，即将句子拆解为单独的词元（token，可以为字符或者单词）。一般在机器翻译类任务中，我们习惯进行单词级词元化，即每个词元要么为一个单词，要么为一个标点符号。同一个单词，不论首字母是否大写，都应该对应同一个词元，故在分词前，我们需统一将单词转换为小写。

```text
"Hello world!" --> ["hello", "world", "!"]
```

接下来，我们创建数据加载器`Multi30K`。后期调用该类进行遍历时，每次返回当前源语言（德语）与目标语言（英语）文本描述的词元列表。

In [32]:
class Multi30K():
    """Multi30K数据集加载器

    加载Multi30K数据集并处理为一个Python迭代对象。

    """
    def __init__(self, path):
        self.data = self._load(path)
        
    def _load(self, path):
        def tokenize(text):
            # 对句子进行分词，统一大小写
            text = text.rstrip()
            return [tok.lower() for tok in re.findall(r'\w+|[^\w\s]', text)]
        
        # 读取Multi30K数据，并进行分词
        members = {i.split('.')[-1]: i for i in os.listdir(path)}
        de_path = os.path.join(path, members['de'])
        en_path = os.path.join(path, members['en'])
        with open(de_path, 'r', encoding='utf-8') as de_file:
            de = de_file.readlines()[:-1]
            de = [tokenize(i) for i in de]
        with open(en_path, 'r', encoding='utf-8') as en_file:
            en = en_file.readlines()[:-1]
            en = [tokenize(i) for i in en]

        return list(zip(de, en))
        
    def __getitem__(self, idx):
        return self.data[idx]
    
    def __len__(self):
        return len(self.data)

In [33]:
train_dataset, valid_dataset, test_dataset = Multi30K(train_path), Multi30K(valid_path), Multi30K(test_path)

In [34]:
## test
# 对解压和分词结果进行测试，打印测试数据集第一组英德语文本，
# 可以看到每一个单词和标点符号已经被单独分离出来
for de, en in test_dataset:
    print(f'de = {de}')
    print(f'en = {en}')
    break

de = ['ein', 'mann', 'mit', 'einem', 'orangefarbenen', 'hut', ',', 'der', 'etwas', 'anstarrt', '.']
en = ['a', 'man', 'in', 'an', 'orange', 'hat', 'starring', 'at', 'something', '.']


### 构建单词词典

将每个词元映射到从0开始的数字索引中（为节约存储空间，可过滤掉词频低的词元），词元和数字索引所构成的集合叫做词典（vocabulary）。

以上述“Hello world!”为例，该序列组成的词典为：

```text
{"<unk>": 0, "<pad>": 1, "<bos>": 2, "<eos>": 3, "hello": 4, "world": 5, "!": 6}
```

在构建词典中，我们使用了4个特殊词元。

- `<unk>`：未知词元（unknown），将出现次数少于一定频率的单词统一判定为未知词元；
- `<bos>`：起始词元（begin of sentence），用来标注一个句子的开始；
- `<eos>`：结束词元（end of sentence），用来标注一个句子的结束；
- `<pad>`：填充词元（padding），当句子长度不够时将句子填充至统一长度；

通过`Vocab`创建词典后，我们可以实现词元与数字索引之间的互相转换。我们可以通过调用`enocde`函数，返回输入词元或者词元序列对应的数字索引或数字索引序列，反之亦然，我们同样可以通过调用`decode`函数，返回输入数字索引或数字索引序列对应的词元或词元序列。


In [35]:
class Vocab:
    """通过词频字典，构建词典"""

    special_tokens = ['<unk>', '<pad>', '<bos>', '<eos>']

    def __init__(self, word_count_dict, min_freq=1):
        self.word2idx = {}
        for idx, tok in enumerate(self.special_tokens):
            self.word2idx[tok] = idx

        # 过滤低词频的词元，并为每个词元配置数字索引
        filted_dict = {
            w: c
            for w, c in word_count_dict.items() if c >= min_freq
        }
        for w, _ in filted_dict.items():
            self.word2idx[w] = len(self.word2idx)

        self.idx2word = {idx: word for word, idx in self.word2idx.items()}

        self.bos_idx = self.word2idx['<bos>']  # 特殊占位符：序列开始
        self.eos_idx = self.word2idx['<eos>']  # 特殊占位符：序列结束
        self.pad_idx = self.word2idx['<pad>']  # 特殊占位符：补充字符
        self.unk_idx = self.word2idx['<unk>']  # 特殊占位符：低词频词元或未曾出现的词元

    def _word2idx(self, word):
        """单词映射至数字索引"""
        if word not in self.word2idx:
            return self.unk_idx
        return self.word2idx[word]

    def _idx2word(self, idx):
        """数字索引映射至单词"""
        if idx not in self.idx2word:
            raise ValueError('input index is not in vocabulary.')
        return self.idx2word[idx]

    def encode(self, word_or_list):
        """将单个单词或单词数组映射至单个数字索引或数字索引数组"""
        if isinstance(word_or_list, list):
            return [self._word2idx(i) for i in word_or_list]
        return self._word2idx(word_or_list)

    def decode(self, idx_or_list):
        """将单个数字索引或数字索引数组映射至单个单词或单词数组"""
        if isinstance(idx_or_list, list):
            return [self._idx2word(i) for i in idx_or_list]
        return self._idx2word(idx_or_list)

    def __len__(self):
        return len(self.word2idx)

In [36]:
# test
word_count = {'a':20, 'b':10, 'c':1, 'd':2}

vocab = Vocab(word_count, min_freq=2)
len(vocab)

7

通过自定义词频字典进行测试，我们可以看到词典已去除词频少于2的词元c，并加入了默认的四个特殊占位符，故词典整体长度为：4 - 1 + 4 = 7

```
word_count = {'a':20, 'b':10, 'c':1, 'd':2}

vocab = Vocab(word_count, min_freq=2)
len(vocab)
```

使用`collections`中的`Counter`和`OrderedDict`统计英/德语每个单词在整体文本中出现的频率。构建词频字典，然后再将词频字典转为词典。其中，收录所有源语言（德语）词元的词典为`de_vocab`，收录所有目标语言（英语）词元的词典为`en_vocab`。

在分配数字索引时有一个小技巧：常用的词元对应数值较小的索引，这样可以节约空间。

In [37]:
from collections import Counter, OrderedDict

def build_vocab(dataset):
    de_words, en_words = [], []
    for de, en in dataset:
        de_words.extend(de)
        en_words.extend(en)

    de_count_dict = OrderedDict(sorted(Counter(de_words).items(), key=lambda t: t[1], reverse=True))
    en_count_dict = OrderedDict(sorted(Counter(en_words).items(), key=lambda t: t[1], reverse=True))

    return Vocab(de_count_dict, min_freq=2), Vocab(en_count_dict, min_freq=2)

In [38]:
de_vocab, en_vocab = build_vocab(train_dataset)
print('Unique tokens in de vocabulary:', len(de_vocab))

Unique tokens in de vocabulary: 7882


### 数据迭代器

数据预处理的最后一步是创建数据迭代器。截至目前，我们已经通过数据加载器`Multi30K`将源语言（德语）与目标语言（英语）的文本描述转换为词元序列，并构建了词元与数字索引一一对应的词典，接下来，需要将词元序列转换为数字索引序列。

还是以“Hello world!”为例，我们逐步演示数据迭代器中的操作

1. 我们将表示开始和结束的特殊词元`<bos>`和`<eos>`分别添加在每个词元序列的句首和句尾。

```text
["hello", "world", "!"] --> ["<bos>", "hello", "world", "!", "<eos>"]
```

2. 统一序列长度（超出长度的进行截断，未达到长度的通过填充`<pad>`进行补齐）,同时记录序列的有效长度。此处假定统一的长度为7。

```text
["<bos>", "hello", "world", "!", "<eos>"] --> ["<bos>", "hello", "world", "!", "<eos>", "<pad>", "<pad>"]， valid length = 5
```

3. 最后，对文本序列进行批处理。对于每个batch中的序列，通过调用词典中的`encode`为序列中的所有词元找到其对应的数字索引，将结果以`Tensor`的形式返回。

```text
["<bos>", "hello", "world", "!", "<eos>", "<pad>", "<pad>"] --> [2, 4, 5, 6, 3, 1, 1] --> tensor
```

In [39]:
class Iterator():
    """创建数据迭代器"""
    def __init__(self, dataset, de_vocab, en_vocab, batch_size, max_len=32, drop_reminder=False):
        self.dataset = dataset
        self.de_vocab = de_vocab
        self.en_vocab = en_vocab

        self.batch_size = batch_size
        self.max_len = max_len
        self.drop_reminder = drop_reminder

        length = len(self.dataset) // batch_size
        self.len = length if drop_reminder else length + 1  # 批量数量

    def __call__(self):
        def pad(idx_list, vocab, max_len):
            """统一序列长度，并记录有效长度"""
            idx_pad_list, idx_len = [], []
            # 当前序列度超过最大长度时，将超出的部分丢弃；当前序列长度小于最大长度时，用占位符补齐
            for i in idx_list:
                if len(i) > max_len - 2:
                    idx_pad_list.append(
                        [vocab.bos_idx] + i[:max_len-2] + [vocab.eos_idx]
                    )
                    idx_len.append(max_len)
                else:
                    idx_pad_list.append(
                        [vocab.bos_idx] + i + [vocab.eos_idx] + [vocab.pad_idx] * (max_len - len(i) - 2)
                    )
                    idx_len.append(len(i) + 2)
            return idx_pad_list, idx_len

        def sort_by_length(src, trg):
            """对德/英语的字段长度进行排序"""
            data = zip(src, trg)
            data = sorted(data, key=lambda t: len(t[0]), reverse=True)
            return zip(*list(data))

        def encode_and_pad(batch_data, max_len):
            """将批量中的文本数据转换为数字索引，并统一每个序列的长度"""
            # 将当前批量数据中的词元转化为索引
            src_data, trg_data = zip(*batch_data)
            src_idx = [self.de_vocab.encode(i) for i in src_data]
            trg_idx = [self.en_vocab.encode(i) for i in trg_data]

            # 统一序列长度
            src_idx, trg_idx = sort_by_length(src_idx, trg_idx)
            src_idx_pad, src_len = pad(src_idx, de_vocab, max_len)
            trg_idx_pad, _ = pad(trg_idx, en_vocab, max_len)

            return src_idx_pad, src_len, trg_idx_pad

        for i in range(self.len):
            # 获取当前批量的数据
            if i == self.len - 1 and not self.drop_reminder:
                batch_data = self.dataset[i * self.batch_size:]
            else:
                batch_data = self.dataset[i * self.batch_size: (i+1) * self.batch_size]

            src_idx, src_len, trg_idx = encode_and_pad(batch_data, self.max_len)
            # 将序列数据转换为tensor
            yield mindspore.Tensor(src_idx, mindspore.int32), \
                mindspore.Tensor(src_len, mindspore.int32), \
                mindspore.Tensor(trg_idx, mindspore.int32)

    def __len__(self):
        return self.len

In [40]:
train_iterator = Iterator(train_dataset, de_vocab, en_vocab, batch_size=128, max_len=32, drop_reminder=True)
valid_iterator = Iterator(valid_dataset, de_vocab, en_vocab, batch_size=128, max_len=32, drop_reminder=False)
test_iterator = Iterator(test_dataset, de_vocab, en_vocab, batch_size=1, max_len=32, drop_reminder=False)

## 2. 模型搭建

In [41]:
# vocabulary
src_vocab_size = len(de_vocab)
trg_vocab_size = len(en_vocab)
src_pad_idx = de_vocab.pad_idx
trg_pad_idx = en_vocab.pad_idx

# hyper-parameters
d_model = 512
d_ff = 2048
n_layers = 6
n_heads = 8

# 实例化模型
encoder = Encoder(src_vocab_size, d_model, n_heads, d_ff, n_layers, dropout_p=0.1)
decoder = Decoder(trg_vocab_size, d_model, n_heads, d_ff, n_layers, dropout_p=0.1)
model = Transformer(encoder, decoder)

continue_training =False
if continue_training:
    # 加载之前训练好的模型
    param_dict = load_checkpoint('saved_models/transformer_20240326213006.ckpt')
    load_param_into_net(model, param_dict)


## 3. 模型训练

- 损失函数：定义如何计算模型输出(logits)与目标(targets)之间的误差，这里可以使用交叉熵损失（CrossEntropyLoss）

- 优化器：MindSpore将模型优化算法的实现称为**优化器**。优化器内部定义了模型的参数优化过程（即梯度如何更新至模型参数），所有优化逻辑都封装在优化器对象中。

In [42]:
loss_fn = nn.CrossEntropyLoss(ignore_index=trg_pad_idx)
optimizer = nn.Adam(model.trainable_params(), learning_rate=0.0001)

### 前向网络计算逻辑

在训练过程中，表示句子结尾的\<eos\>占位符应是被模型预测出来，而不是作为模型的输入，所以在处理Decoder的输入时，我们需要移除目标序列最末的\<eos\>占位符。

$$\text{trg} = [\text{<bos>}, x_1, x_2, ..., x_n, \text{<eos>}]$$
$$\text{trg[:-1]} = [\text{<bos>}, x_1, x_2, ..., x_n]$$

其中，$x_i$代表目标序列中第i个表示实际内容的词元。

我们期望最终的输出包含表示句末的\<eos\>，不包含表示句首的\<bos\>，所以在计算损失时，需要同样去除的目标序列的句首\<bos\>占位符，再进行比较。

$$\text{output} = [y_1, y_2, ..., y_n, \text{<eos>}]$$
$$\text{trg[1:]} = [x_1, x_2, ..., x_n, \text{<bos>}]$$

其中，$y_i$表示预测的第i个实际内容词元。


In [43]:
def forward(enc_inputs, dec_inputs):
    """前向网络
    enc_inputs: [batch_size, src_len]
    dec_inputs: [batch_size, trg_len]
    """
    # 训练过程中不应该包含目标序列中的最后一个词元<eos>
    # logits: [batch_size * (trg_len - 1), trg_vocab_size]
    logits, _, _, _ = model(enc_inputs, dec_inputs[:, :-1], src_pad_idx, trg_pad_idx)
    
    # 推理结果不应该包含目标序列中的第一个词元<bos>
    # targets: [batch_size * (trg_len -1), ]
    targets = dec_inputs[:, 1:].view(-1)
    loss = loss_fn(logits, targets)

    return loss

定义梯度计算函数。

为了优化模型参数，需要求参数对loss的导数。我们调用`mindspore.ops.value_and_grad`函数，来获得function的微分函数。
常用到的参数有三种：

- fn：待求导的函数；
- grad_position：指定求导输入位置的索引；
- weights：指定求导的参数；

由于使用Cell封装神经网络模型，模型参数为Cell的内部属性，此时我们不需要使用`grad_position`指定对函数输入求导，因此将其配置为None。对模型参数求导时，我们使用weights参数，使用`model.trainable_params()`方法从Cell中取出可以求导的参数。

In [44]:
# 反向传播计算梯度
grad_fn = ops.value_and_grad(forward, None, optimizer.parameters)

### 一个训练步的逻辑

In [45]:
# 训练一个step的逻辑
def train_step(enc_inputs, dec_inputs):
    # 反向传播，获得梯度
    loss, grads = grad_fn(enc_inputs, dec_inputs)
    # 权重更新
    optimizer(grads)
    return loss

### 整体逻辑

train

In [46]:
def train(iterator, epoch=0):
    model.set_train(True)
    num_batches = len(iterator)
    total_loss = 0  # 所有batch训练loss的累加
    total_steps = 0  # 训练步数

    with tqdm(total=num_batches) as t:
        t.set_description(f'Epoch: {epoch}')
        for src, src_len, trg in iterator():
            # 计算当前batch数据的loss
            loss = train_step(src, trg)
            total_loss += loss.asnumpy()
            total_steps += 1
            # 当前的平均loss
            curr_loss = total_loss / total_steps
            t.set_postfix({'loss': f'{curr_loss:.2f}'})
            t.update(1)

    return total_loss / total_steps

evaluate

In [47]:
def evaluate(iterator):
    # 在评估中，仅需正向计算loss，无需更新模型参数,故模型状态需设置为训练model.set_train(False)
    model.set_train(False)
    num_batches = len(iterator)
    total_loss = 0  # 所有batch训练loss的累加
    total_steps = 0  # 训练步数

    with tqdm(total=num_batches) as t:
        for src, _, trg in iterator():
            # 计算当前batch数据的loss
            loss = forward(src, trg)
            total_loss += loss.asnumpy()
            total_steps += 1
            # 当前的平均loss
            curr_loss = total_loss / total_steps
            t.set_postfix({'loss': f'{curr_loss:.2f}'})
            t.update(1)

    return total_loss / total_steps

## 4. 训练，启动！

In [ ]:
## 注意，先用一个epoch测试一下，看看模型是否能够正常训练，之后，改为10个左右的epoch即可
num_epochs = 1  # 训练迭代数
best_valid_loss = float('inf')  # 当前最佳验证损失
cache_dir = './saved_models'  # 模型保存路径
ckpt_file_name = os.path.join(cache_dir, 'transformer.ckpt')  # 模型保存路径


for i in range(num_epochs):
    # 模型训练，网络权重更新
    train_loss = train(train_iterator, i)
    # 网络权重更新后对模型进行验证
    valid_loss = evaluate(valid_iterator)
    # 保存当前效果最好的模型
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        save_checkpoint(model, ckpt_file_name)

如果训练正常，你会看到类似的结果。（本机配置Ubuntu 20.04，GPU为RTX 3090）
![](assets/images/2024-03-26-21-19-02.png)

## 5. 模型预测

首先，通过`load_checkpoint`与`load_param_into_net`将训练好的模型参数加载入新实例化的模型中。

In [ ]:
# 实例化新模型
encoder = Encoder(src_vocab_size, d_model, n_heads, d_ff, n_layers, dropout_p=0.1)
decoder = Decoder(trg_vocab_size, d_model, n_heads, d_ff, n_layers, dropout_p=0.1)
new_model = Transformer(encoder, decoder)

# 加载之前训练好的模型
param_dict = load_checkpoint(ckpt_file_name)
load_param_into_net(new_model, param_dict)

推理过程中无需对模型参数进行更新，所以这里`model.set_train(False)`。

我们输入一个德文语句，期望可以返回翻译好的英文语句。

首先通过Encoder提取德文序列中的特征信息，并将其传输至Decoder。

Decoder最开始的输入为起始占位符\<bos\>，每次会根据输入预测下一个出现的单词，并对输入进行更新，直到预测出终止占位符\<eos\>。

In [ ]:
def inference(sentence, max_len=32):
    """模型推理：输入一个德语句子，输出翻译后的英文句子
    enc_inputs: [batch_size(1), src_len]
    """
    new_model.set_train(False)

    # 对输入句子进行分词
    if isinstance(sentence, str):
        tokens = [tok.lower() for tok in re.findall(r'\w+|[^\w\s]', sentence.rstrip())]
    else:
        tokens = [token.lower() for token in sentence]
    
    # 补充起始、终止占位符，统一序列长度
    if len(tokens) > max_len - 2:
        src_len = max_len
        tokens = ['<bos>'] + tokens[:max_len - 2] + ['<eos>']
    else:
        src_len = len(tokens) + 2
        tokens = ['<bos>'] + tokens + ['<eos>'] + ['<pad>'] * (max_len - src_len)

    # 将德语单词转换为数字索引，并进一步转换为tensor
    # enc_inputs: [1, src_len]
    indexes = de_vocab.encode(tokens)
    enc_inputs = Tensor(indexes, mstype.float32).expand_dims(0)
    
    # 将输入送入encoder，获取信息
    enc_outputs, _ = new_model.encoder(enc_inputs, src_pad_idx)

    # 初始化decoder输入，此时仅有句首占位符<pad>
    # dec_inputs: [1, 1]
    dec_inputs = Tensor([[en_vocab.bos_idx]], mstype.float32)

    max_len = enc_inputs.shape[1]
    for _ in range(max_len):
        # dec_outputs: [batch_size(1) * len(dec_inputs), trg_vocab_size]
        dec_outputs, _, _ = new_model.decoder(dec_inputs, enc_inputs, enc_outputs, src_pad_idx, trg_pad_idx)
        dec_logits = dec_outputs.view((-1, dec_outputs.shape[-1]))

        # 找到下一个词的概率分布，并输出预测
        # dec_logits: [1, trg_vocab_size]
        # pred: [1, 1]
        dec_logits = dec_logits[-1, :]
        pred = dec_logits.argmax(axis=0).expand_dims(0).expand_dims(0)
        pred = pred.astype(mstype.float32)

        # 更新dec_inputs
        dec_inputs = ops.concat((dec_inputs, pred), axis=1)

        # 如果出现<eos>，则终止循环
        if int(pred.asnumpy()[0]) == en_vocab.eos_idx:
            break

    # 将数字索引转换为英文单词
    trg_indexes = [int(i) for i in dec_inputs.view(-1).asnumpy()]
    eos_idx = trg_indexes.index(en_vocab.eos_idx) if en_vocab.eos_idx in trg_indexes else -1
    trg_tokens = en_vocab.decode(trg_indexes[1:eos_idx])

    return trg_tokens

In [ ]:
# test
example_idx = 0

src = test_dataset[example_idx][0]
trg = test_dataset[example_idx][1]
pred_trg = inference(src)

print(f'src = {src}')
print(f'trg = {trg}')
print(f"predicted trg = {pred_trg}")

### 评估预测好坏的指标

双语替换评测得分（bilingual evaluation understudy，BLEU）为衡量文本翻译模型生成出来的语句好坏的一种算法，它的核心在于评估机器翻译的译文 $\text{pred}$ 与人工翻译的参考译文 $\text{label}$ 的相似度。通过对机器译文的片段与参考译文进行比较，计算出各个片段的的分数，并配以权重进行加和，基本规则为：

1. 惩罚过短的预测，即如果机器翻译出来的译文相对于人工翻译的参考译文过于短小，则命中率越高，需要施加更多的惩罚；
2. 对长段落匹配更高的权重，即如果出现长段落的完全命中，说明机器翻译的译文更贴近人工翻译的参考译文；

BLEU的公式如下：

$$exp(min(0, 1-\frac{len(\text{label})}{len(\text{pred})})\Pi^k_{n=1}p_n^{1/2^n})$$

- `len(label)`：人工翻译的译文长度
- `len(pred)`：机器翻译的译文长度
- `p_n`：n-gram的精度

本实验中，我们使用mindspore的`BleuScore()`类来计算模型的BLEU得分。

In [ ]:
def calculate_bleu(dataset, max_len=50):
    bleu_metric = BleuScore()

    candidate_corpus = []  # 机器翻译语料库列表
    reference_corpus = []  # 引用语料库列表，即真实的目标语句列表

    for data in tqdm(dataset[:20]):
        src = data[0]  # 源语句：德语
        trg = data[1]  # 目标语句：英语
        pred_trg = inference(src, max_len)  # 获取模型预测结果
        candidate_corpus.append(pred_trg)  # 将模型预测结果加入机器翻译语料库列表
        reference_corpus.append([trg])  # 将真实目标语句加入引用语料库列表

    # 更新BleuScore实例
    bleu_metric.clear()
    bleu_metric.update(candidate_corpus, reference_corpus)

    # 计算BLEU分数
    bleu_score = bleu_metric.eval()
    return bleu_score
# 计算BLEU Score
bleu_score = calculate_bleu(test_dataset)

print(f'BLEU score = {bleu_score*100:.2f}')


## 附录

[1]原文的解释为：“While for small values of dk the two mechanisms perform similarly, additive attention outperforms dot product attention without scaling for larger values of dk. We suspect that for large values of dk, the dot products grow large in magnitude, pushing the softmax function into regions where it has extremely small gradients 4. To counteract this effect, we scale the dot products by 1 √dk .”